In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_tweet=pd.read_excel('/content/dev_data_tweet.xlsx')
data_tweet=data_tweet[data_tweet['Mobile_Tech_Tag']==1]

In [ ]:
#data_tweet

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import string
from nltk.tokenize.toktok import ToktokTokenizer
import unicodedata

#!pip install emoji
import emoji
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [ ]:
def remove_punct(texts):
  punkts='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
  lis=[]
  for char in punkts:
    lis.append(char)
   
  

  no_punct=''
  for c in texts:
    if(c not in lis):
      no_punct+=c
  
  return no_punct.strip()

In [ ]:
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [ ]:
#@title Contraction_Mapping (Run this)

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
def expand_contractions(text, contraction_mapping=contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [ ]:
from bs4 import BeautifulSoup
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
def demojize(text):
  tweet = emoji.demojize(text)
  tweet = tweet.replace(":"," ")
  tweet = ' '.join(tweet.split())
  return(tweet)

In [ ]:
def remove_https(text):
  sent=''
  for word in text.split():
    if(word[:5]=='https'):
      continue
    else:
      sent+=' '+word
  return(sent.strip())    


In [ ]:
def remove_handle_names(text):
  sent=''
  for word in text.split():
    if(word[0]=='@' or word=='QT' or word=='RT'):
      continue
    else:
      sent+=' '+word

  return(sent.strip())  

In [ ]:
#@title Not required

def change_contractions(text):
  sent=''
  for word in text.split():
    if(word.lower() in contraction_mapping.keys()):
      sent+=" "+contraction_mapping[word.lower()]
    else:
      sent+=' '+word.lower()
  
  return(sent.strip())      

def word_lemmat(text):
  lemma_text=[lemmatizer.lemmatize(i) for i in text]
  return  lemma_text

def word_stemmer(text):
  stem_text=[stemmer.stem(i) for i in text]
  return  stem_text  

In [ ]:
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:remove_handle_names(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:strip_html_tags(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:expand_contractions(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:remove_punct(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:remove_stopwords(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:demojize(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:remove_punct(x))
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:remove_https(x))

In [ ]:
data_tweet
#data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x:demojize(x))

,Tweet_ID,Tweet,Mobile_Tech_Tag
3000,tweet_3001,Samsung making another addition series word up...,1
3001,tweet_3002,Want hands GalaxyS21 pleadingface Samsung Gala...,1
3002,tweet_3003,next level camera starstruck GalaxyS21 Samsung...,1
3003,tweet_3004,Trick get Google Home Mini 499 Without Timespr...,1
3004,tweet_3005,pioneers 64MP Smartphone Camera technology Ind...,1
...,...,...,...
3995,tweet_3996,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3996,tweet_3997,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3998,tweet_3999,Aapne phone hi galat choose KR liya haisorry s...,1


In [ ]:
#data_tweet.to_csv('Cleaned_Mobile_tech_file.csv')

In [ ]:
data_tweet

,Tweet_ID,Tweet,Mobile_Tech_Tag
3000,tweet_3001,Samsung making another addition series word up...,1
3001,tweet_3002,Want hands GalaxyS21 pleadingface Samsung Gala...,1
3002,tweet_3003,next level camera starstruck GalaxyS21 Samsung...,1
3003,tweet_3004,Trick get Google Home Mini 499 Without Timespr...,1
3004,tweet_3005,pioneers 64MP Smartphone Camera technology Ind...,1
...,...,...,...
3995,tweet_3996,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3996,tweet_3997,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1
3998,tweet_3999,Aapne phone hi galat choose KR liya haisorry s...,1


In [ ]:
#@title Normalize corpus

def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=False, 
                     text_lemmatization=False, special_char_removal=False, 
                     stopword_removal=True, remove_digits=False):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [ ]:
#!pip install textblob
from textblob import TextBlob

In [ ]:
def convert_Hinglish_to_Hindi(text):
  sent=''
  for word in text.split():
    b = TextBlob(word)
    try:
      lang=b.detect_language()
      if(lang=='en'):
        sent+=' '+transliterate(word, sanscript.ITRANS, sanscript.DEVANAGARI)
      else:
        sent+=' '+word
    except:
      #print(word)
      sent+=' '+word
      continue  
    
  return(sent.strip())      

In [ ]:
#data_tweet['Tweet'][3500]

In [ ]:
def identify_hinglish(text):
  count=0
  for word in text.split():
    b=TextBlob(word)
    try:
      lang=b.detect_language()
      if(lang=='hi'):
        return(1)
      else:
        count+=1
        if(count>=5):
          return(0)  
    except:
      continue
  return(0)      

In [ ]:
data_tweet['Hinglish_tag']=data_tweet['Tweet'].apply(lambda x: identify_hinglish(x))

In [ ]:
def convert_all_sent_to_english(text,is_hinglish=1):
  if(is_hinglish):
    sent=(convert_Hinglish_to_Hindi(text))
    hi_blob = TextBlob(sent)
    return(hi_blob.translate(to='en'))
  else:
    return(text)  

In [ ]:
#!pip install aspect-based-sentiment-analysis
import aspect_based_sentiment_analysis as absa

In [ ]:
nlp = absa.load()
text = data_tweet['Tweet'][0]
slack,price = nlp(text, aspects=['Apple','Samsung'])

In [ ]:
slack.sentiment,price.sentiment,

(<Sentiment.negative: 1>, <Sentiment.positive: 2>)

In [ ]:
lis=[]
for i in range(3832,len(data_tweet)):
  #if(identify_hinglish(data_tweet['Tweet'][i])):
  lis.append(convert_all_sent_to_english(data_tweet['Tweet'][i]))
  
  print(i)

In [ ]:
data_tweet['Tweet']=data_tweet['Tweet'].apply(lambda x: convert_all_sent_to_english(x))

In [ ]:
data_tweet.to_csv('Final_cleaned.csv')

In [ ]:
convert_Hinglish_to_Hindi(data_tweet['Tweet'][3990])

RT
ने
5G
कर
है


'RT ZeeNews चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G स्मार्टफोन लॉन्च कर दिया है स्मर्त्फोने'

In [ ]:
#!pip install indic-transliteration

In [ ]:
# import the module 
from indic_transliteration import sanscript 
from indic_transliteration.sanscript import transliterate 
  
# the text to be transliterated 
text = "Apa sabhii kaa yahaan svaagat hai."
  
# printing the transliterated text 
print(transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)) 

आप सभी का यहान् स्वागत् है।


In [ ]:
#data_tweet['English_Part_of_Tweet']=data_tweet['Tweet'].str.replace('[^A-Za-z0-9]+'," ")
#data_tweet['Numeric_Part_of_Tweet']=data_tweet['Tweet'].str.replace('[^0-9]+'," ")